# Using PYQt5

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QFormLayout, QLineEdit, QPushButton, QTextEdit, QLabel, QTabWidget, QMessageBox
from PyQt5.QtCore import Qt
import sqlite3
import pandas as pd

class InventoryApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Inventory Management System')
        self.setGeometry(100, 100, 800, 600)
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout()
        self.central_widget.setLayout(self.layout)

        self.tab_widget = QTabWidget()
        self.layout.addWidget(self.tab_widget)

        self.inventory_tab = QWidget()
        self.sales_tab = QWidget()
        self.report_tab = QWidget()

        self.tab_widget.addTab(self.inventory_tab, 'Inventory')
        self.tab_widget.addTab(self.sales_tab, 'Sales')
        self.tab_widget.addTab(self.report_tab, 'Reports')

        self.setup_inventory_tab()
        self.setup_sales_tab()
        self.setup_report_tab()
        self.create_db()

    def create_db(self):
        try:
            conn = sqlite3.connect('inventory_sales.db')
            cursor = conn.cursor()
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS Products (
                product_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                quantity INTEGER,
                price REAL
            )
            """)
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS Sales (
                sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
                product_id INTEGER,
                quantity INTEGER,
                sale_date DATE,
                FOREIGN KEY (product_id) REFERENCES Products(product_id)
            )
            """)
            conn.commit()
            conn.close()
        except Exception as e:
            self.show_error_message("Database Error", str(e))

    def setup_inventory_tab(self):
        layout = QFormLayout()

        self.product_name = QLineEdit()
        self.quantity = QLineEdit()
        self.price = QLineEdit()

        layout.addRow(QLabel("Product Name:"), self.product_name)
        layout.addRow(QLabel("Quantity:"), self.quantity)
        layout.addRow(QLabel("Price:"), self.price)

        self.add_button = QPushButton("Add Product")
        self.add_button.clicked.connect(self.add_product)

        layout.addWidget(self.add_button)
        self.inventory_tab.setLayout(layout)

    def setup_sales_tab(self):
        layout = QFormLayout()

        self.sale_product_id = QLineEdit()
        self.sale_quantity = QLineEdit()
        self.sale_date = QLineEdit()

        layout.addRow(QLabel("Product ID:"), self.sale_product_id)
        layout.addRow(QLabel("Quantity:"), self.sale_quantity)
        layout.addRow(QLabel("Sale Date (YYYY-MM-DD):"), self.sale_date)

        self.record_sale_button = QPushButton("Record Sale")
        self.record_sale_button.clicked.connect(self.record_sale)

        layout.addWidget(self.record_sale_button)
        self.sales_tab.setLayout(layout)

    def setup_report_tab(self):
        self.report_text = QTextEdit()
        self.report_text.setReadOnly(True)

        self.report_button = QPushButton("Generate Sales Report")
        self.report_button.clicked.connect(self.generate_report)

        layout = QVBoxLayout()
        layout.addWidget(self.report_button)
        layout.addWidget(self.report_text)
        self.report_tab.setLayout(layout)

    def add_product(self):
        try:
            name = self.product_name.text()
            quantity = int(self.quantity.text())
            price = float(self.price.text())
            if not name or quantity < 0 or price < 0:
                raise ValueError("Invalid input values")
            conn = sqlite3.connect('inventory_sales.db')
            cursor = conn.cursor()
            cursor.execute("INSERT INTO Products (name, quantity, price) VALUES (?, ?, ?)",
                            (name, quantity, price))
            conn.commit()
            conn.close()
        except ValueError as ve:
            self.show_warning_message("Input Error", str(ve))
        except Exception as e:
            self.show_error_message("Database Error", str(e))

    def record_sale(self):
        try:
            product_id = int(self.sale_product_id.text())
            quantity = int(self.sale_quantity.text())
            sale_date = self.sale_date.text()
            if quantity < 0:
                raise ValueError("Quantity cannot be negative")
            conn = sqlite3.connect('inventory_sales.db')
            cursor = conn.cursor()
            cursor.execute("INSERT INTO Sales (product_id, quantity, sale_date) VALUES (?, ?, ?)",
                            (product_id, quantity, sale_date))
            cursor.execute("UPDATE Products SET quantity = quantity - ? WHERE product_id = ?",
                            (quantity, product_id))
            conn.commit()
            conn.close()
        except ValueError as ve:
            self.show_warning_message("Input Error", str(ve))
        except sqlite3.IntegrityError:
            self.show_warning_message("Data Error", "Product ID not found")
        except Exception as e:
            self.show_error_message("Database Error", str(e))

    def generate_report(self):
        try:
            conn = sqlite3.connect('inventory_sales.db')
            query = """
            SELECT p.name, SUM(s.quantity) as total_quantity, SUM(s.quantity * p.price) as total_sales
            FROM Sales s
            JOIN Products p ON s.product_id = p.product_id
            GROUP BY p.name
            """
            df = pd.read_sql_query(query, conn)
            conn.close()
            self.report_text.setPlainText(df.to_string(index=False))
        except Exception as e:
            self.show_error_message("Report Error", str(e))

    def show_warning_message(self, title, message):
        msg = QMessageBox()
        msg.setIcon(QMessageBox.Warning)
        msg.setWindowTitle(title)
        msg.setText(message)
        msg.exec_()

    def show_error_message(self, title, message):
        msg = QMessageBox()
        msg.setIcon(QMessageBox.Critical)
        msg.setWindowTitle(title)
        msg.setText(message)
        msg.exec_()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = InventoryApp()
    window.show()
    sys.exit(app.exec_())


SystemExit: 0

D:\jupyter\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
import datetime

now = datetime.datetime.today().strftime('%d/%m/%Y')

# currentDate = datetime.datetime.strptime('01/08/2015','%d/%m/%Y').date()

In [19]:
# now.date().isoformat()
type(now)

str

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QComboBox, QLabel, QVBoxLayout

def main():
    app = QApplication(sys.argv)
    window = QWidget()
    window.setWindowTitle("Combo Box Example")
    window.setGeometry(100, 100, 400, 200)  # (x, y, width, height)

    layout = QVBoxLayout()

    # Create a QComboBox widget
    combo_box = QComboBox(window)
    combo_box.addItem("Python")
    combo_box.addItem("Java")
    combo_box.addItem("C++")
    layout.addWidget(combo_box)

    # Create a QLabel to display the selected item
    label = QLabel("Selected Item: ", window)
    layout.addWidget(label)

    # Connect the combo box's currentIndexChanged signal to update_label slot
    def update_label():
        selected_item = combo_box.currentText()
        label.setText("Selected Item: " + selected_item)

    combo_box.currentIndexChanged.connect(update_label)

    window.setLayout(layout)
    window.show()

    sys.exit(app.exec_())

if __name__ == "__main__":
    main()

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QComboBox, QLabel, QVBoxLayout

def main():
    app = QApplication(sys.argv)
    window = QWidget()
    window.setWindowTitle("Combo Box Example")
    window.setGeometry(100, 100, 400, 200)  # (x, y, width, height)

    layout = QVBoxLayout()

    # Create a QComboBox widget
    combo_box = QComboBox(window)
    combo_box.addItem("Python")
    combo_box.addItem("Java")
    combo_box.addItem("C++")
    layout.addWidget(combo_box)

    # Create a QLabel to display the selected item
    label = QLabel("Selected Item: ", window)
    layout.addWidget(label)

    # Connect the combo box's currentIndexChanged signal to update_label slot
    def update_label():
        selected_item = combo_box.currentText()
        label.setText("Selected Item: " + selected_item)

    combo_box.currentIndexChanged.connect(update_label)

    window.setLayout(layout)
    window.show()

    sys.exit(app.exec_())

if __name__ == "__main__":
    main()

SystemExit: 0

D:\jupyter\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
